<table style="width: 100%;" id="nb-header>">
        <tr style="background-color: transparent;"><td>
            <img src="https://ds-connectors.github.io/econ-fa19/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
        </td><td>
            <p style="text-align: right; font-size: 10pt;"><strong>Economic Models</strong>, Spring 2020<br>
                Dr. Eric Van Dusen<br>
            Notebook by Chris Pyles</p></td></tr>
    </table>

<!-- BEGIN QUESTION -->

# Project 2: Game Theory and Behavioral Economics

<!-- END QUESTION -->

In this notebook, we will introduce a fundamental thought experiment in the economics and statistics subdomain of **game theory**, called the Prisoner's Dilemma. We will then extend our consideration of game theory and behavioral economics to look at a recent paper that explores a phenomenon at the intersection of these topics.

At the end of this project, you should:
1. Understand the prisoner's dilemma and strategies for playing the game
3. Understand the game theory underpinnings of the Thomas and Pemstein (2015) paper
4. Be able to rerun the analysis in the aforementioned paper using hypothesis testing

In [57]:
from datascience import *
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-muted')
plt.rcParams['figure.figsize'] = [25,10]
from players import *
import otter
grader = otter.Notebook()

## Part 1: The Iterated Prisoner's Dilemma

The [prisoner's dilemma](https://en.wikipedia.org/wiki/Prisoner%27s_dilemma) is a classic game first discussed by Merrill Flood and Melvin Dresher in 1950. In this game, there are two prisoners who have been captured and are being interrogated. The prinsoners cannot contact each other in any way. They have two options: they can **defect** (betray the other prisoner to the police) or they can **cooperate** (maintain their silence). If both defect, both receive 4 years in prinson. If one defects and the other does not, the defector goes free and the cooperator receives 5 years in prison. If both cooperate (meaning neither talks to the police), then they each receive 2 years in prison. The purpose of this game is to consider how a completely rational person would be best advised to proceed, and how different strategies for playing this game can be more or less effective.

<table>
    
<tr style="background-color: white;"><td></td><td></td><td colspan="2">Prisoner A</td></tr>
<tr><td></td><td></td><td>Cooperate</td><td>Defect</td></tr>
<tr style="background-color: white;"><td rowspan="2">Prisoner B</td><td>Cooperate</td><td style="background-color: #F5F5F5;">A: 2, B: 2</td><td>A: 0, B: 5</td></tr>
<tr><td>Defect</td><td>A: 5, B: 0</td><td style="background-color: #F5F5F5;">A: 4, B: 4</td></tr>
    
</table>

In this project, we will study the **iterated prisoner's dilemma**, a game paradigm in which the prisoner's dilemma is played over multiple rounds and in tournaments to determine the best strategy for playing the game. This paradigm was introduced by [Robert Axelrod](https://en.wikipedia.org/wiki/Robert_Axelrod) to use the prisoner's dilemma as a lens through which to study the Cold War during the Cuban Missile Crisis. Axelrod created a tournment out of an iterated prisoner's dilemma and invited theoreticians to write programs that could strategically play the game, and then pitted them one against another in a round-robin-style tournament.

Let's first consider the two most basic strategies: the Cooperator (who always cooperates) and the Defector (who always defects). We define the _winner_ of a match to be the person with the lowest number of prison years accrued.

<!-- BEGIN QUESTION -->

**Question 1.1:** In our match of Cooperator vs. Defector, who do you expect to win (to have the fewest number of years)? Why?

<!--
BEGIN QUESTION
name: q1_1
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



In this project, we will continue using the Python structures we introduced in Lab 9. Here is a short refresher on these classes:

* `Defector`, `Cooperator`, `Random`, `TitForTat`, and `Alternator` are all player classes defined in the lab
* `payoff` returns the payoffs of a single round of play between two players
* `run_match` takes two players and runs an `n` turn match between them, returning the winner or the years accrued during each turn
* `determine winner` takes two players and their mean scores and returns the winning player

Recall our basic match syntax with the 10-turn match below between `Defector` and `Cooperator`.

In [2]:
run_match(Defector(), Cooperator(), n=10)

As you can see, the Defector won the match. This is because at each turn, the Defector defected and the Cooperator cooperated, result in the Cooperator getting 5 years and the Defector getting 0.

Let's now consider some other possible strategies for playing this game over multiple rounds. One possible strategy is to randomly cooperate or defect; this strategy is encapsulated in `Random`. We can optionally pass a probability $p$ (defaults to 0.5) to `Random`, $0 \le p \le 1$, indicating the probability that the player will cooperate; that is, on each turn, `Random(.25)` will have a 25% chance of cooperating.

In [3]:
random_match_winner = run_match(Random(.4), Random(.6))
random_match_winner

<!-- BEGIN QUESTION -->

**Question 1.2:** What are some other possible strategies for playing this game? Give 2 examples.

<!--
BEGIN QUESTION
name: q1_2
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



We provide the player classes described in the table below.

| Name | Description |
|-----|-----|
| `Alternator` | Alternates between C and D |
| `Backstabber` | Forgives first 3 opponent D's then D forever after fourth |
| `Bully` | Starts by defecting and then does the opposite of opponent's previous move |
| `Desperate` | Only cooperates after mutual defection |
| `FoolMeOnce` | Forgives one D then retaliates forever on a second D |
| `Forgiver` | Starts by cooperating however will defect if at any point the opponent has defected more than 10 percent of the time |
| `OnceBitten` | C once on opponent D, but if opponent D's twice in a row defaults to D for 10 turns |
| `TitForTat` | Repeats opponent's previous move |
| `WorseAndWorse` | D with probability $\frac{t}{1000}$, where $t$ is the number of the current turn |

<!-- BEGIN QUESTION -->

**Question 1.3:** Suppose we have a 10-turn match with a backstabber and a desperate. Write down the sequence of defections (D) and cooperations (C) for each player. Who wins the match?

<!--
BEGIN QUESTION
name: q1_3
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



**Question 1.4:** Implement the player `ForgivingTitForTat` who starts by cooperating and then at each turn defects if their opponent has defected more than 10% of the time and their last move was defection. The percentage should be calculated at each turn, so that if the opponent defects to above 10% and then lowers their defection percentage below 10% the `ForgivingTitForTat` player should cooperate.

_Hint:_ Recall our slight change to indexing depending on whether or not the current player is Player 1 in the match. Also note that you can get the number of defections in a player's history by summing it or using `np.count_nonzero`, as defections are recorded as `True`.

<!--
BEGIN QUESTION
name: q1_4
-->

In [ ]:
def forgiving_tit_for_tat_play(self, opponent, is_p1):
    if len(self.history) == 0:
        return ...
    if is_p1:
        return ... and ...
    else:
        return ... and ...
    
ForgivingTitForTat = create_player_class("ForgivingTitForTat", forgiving_tit_for_tat_play)

In [ ]:
grader.check("q1_4")

### Tournaments

Now that we have refamiliarized ourselves with our prisoner's dilemma code from the lab, let's move on to recreating Axelrod's tournament. Recall that Axelrod originally wanted to compare strategies for playing the prisoner's dilemma, which he did by pitting every strategy against every other strategy in a round-robin tournament. In this project, we will do the same thing, albeit on a much smaller scale, with the strategies defined in the table above (and `ForgivingTitForTat`, of course).

<!-- BEGIN QUESTION -->

**Question 1.5:** Which strategy do you think will win our tournament? Why?

<!--
BEGIN QUESTION
name: q1_5
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



Now let's think about runnig the tournament. To do it, we will need to run $\binom{n}{2}$ matches, where $n$ is the number of players in the tournament. Let's start by encapsulating this logic in a function.

**Question 1.6:** Fill in the function `run_tournament` below that takes in an array of players and returns a table with four columns: `p1`, `p1_mean`, `p2`, and `p2_mean`. As in Axelrod's original tournament, use matches with 200 rounds.

_Hint_: Recall how we can use the `winner` argument of `run_match`.

<!--
BEGIN QUESTION
name: q1_6
-->

In [ ]:
def run_tournament(players):
    p1 = make_array()
    p2 = make_array()
    p1_mean = make_array()
    p2_mean = make_array()

    for ...:
        for ...:
            p1_years, p2_years = run_match(...)
            p1 = ...
            p2 = ...
            p1_mean = ...
            p2_mean = ...
            
    results = Table().with_columns(
        "p1", p1,
        "p1_mean", p1_mean,
        "p2", p2,
        "p2_mean", p2_mean
    )

    return results

run_tournament(make_array(Defector(), Cooperator()))

In [ ]:
grader.check("q1_6")

**Question 1.7:** Create a tournament with all players (except defector, cooperator, and random). Assign the results table to `tournament_results`.

<!--
BEGIN QUESTION
name: q1_7
-->

In [ ]:
tournmanet_results = ...
tournmanet_results

In [ ]:
grader.check("q1_7")

We can use a [box plot](https://en.wikipedia.org/wiki/Box_plot) to visualize the scores in our tournaments. A box plot shows the mean score, the outliers, and the min, max, and outer quartiles of the scores. The wide horizontal line in each box indicates the mean score.

In [59]:
sns.boxplot(x="player", y="score", data=flatten_results(tournmanet_results));

<!-- BEGIN QUESTION -->

<!-- BEGIN QUESTION -->

**Question 1.8:** What does the plot above tell you? Do the results coincide with your predictions from Question 1.4? Why or why not?

<!--
BEGIN QUESTION 
name: q1_8
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



Now that we have the experience of creating prisoner's dilemma tournaments, let's consider Axelrod's findings. The [winning deterministic strategy](https://en.wikipedia.org/wiki/Prisoner%27s_dilemma#Strategy_for_the_iterated_prisoner's_dilemma) was, surprisingly, tit-for-tat. This strategy was enetered into the competition by [Anatol Rapoport](https://en.wikipedia.org/wiki/Anatol_Rapoport), coded in only four lines of BASIC. Axelrod studied the best strategies and came up with the following four traits of the biggest winners: niceness, retaliation, forgiveness, and lack of envy.

<!-- BEGIN QUESTION -->

**Question 1.9:** Answer the questions below.

1. Do your results support the conclusion that tit-for-tat is the best deterministic strategy? You should ignore the defector and cooperator strategies.
2. A strategy that is "non-envious" does not strive to score higher than its opponent. Why do you think this trait is linked to good strategies? How does one of the other strategies you looked at (excluding defector, cooperator, and tit-for-tat) embody this trait?

<!--
BEGIN QUESTION
name: q1_9
manual: true
-->

_Type your answer here, replacing this text._

**Question 1.10:** The prisoner's dilemma is often used by economists to study and understand different phenomena that are observed in economies at different scales (e.g. studying oligopolies, advertising practices). A common application of this game is in thinking about combating [**cartels**](https://en.wikipedia.org/wiki/Cartel), groups of market participants that collude with each other to maximize profits by setting prices as a group. Describe how cartels can be viewed as players in an iterated prisoner's dilemma. What are defection and cooperation in this context? What is the score? What is the most desired outcome for consumers?

<!--
BEGIN QUESTION
name: q1_10
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



If you're interested, there's a very interesting [episode of Radiolab](https://www.wnycstudios.org/podcasts/radiolab/segments/104010-one-good-deed-deserves-another) that describes the origins of Axelrod's tournament, the iterated prisoner's dilemma, and the tit-for-tat strategy.

<!-- END QUESTION -->

## Part 2: Height Perceptions

In this section of the project, we are going to move on to replicating the analysis in [this paper](https://doi.org/10.3389/fpsyg.2015.00306) by Thomas and Pemstein (2015). The paper describes two experiments, both of which we will cover in this project.

<div class="alert alert-warning">
    
Take a moment to read the <a href="https://www.frontiersin.org/articles/10.3389/fpsyg.2015.00306/full#h3" target="_blank">Experiment 1 methodology</a> from the paper. Ensure you understand the experiment before moving on.

</div>

**Question 2.1:** The first experiment examined subjects' perception of height by adjusting the

<ol type="A" style="list-style-type: lower-alpha;">
    <li>screen position</li>
    <li>camera position</li>
    <li>camera focus</li>
    <li>camera exposure</li>
</ol>

Assign the letter corresponding to your answer to `q2_1` below.

<!--
BEGIN QUESTION
name: q2_1
points: 1
-->

In [ ]:
q2_1 = ...

In [ ]:
grader.check("q2_1")

Now we'll read in the experimental data from the paper. The table below gives the condition, height estimate, and true height for each subject. Recall that there are two possible experimental conditions: looking down and looking up.

In [20]:
exp1_raw = Table.read_table("frontiers1.csv")
exp1_raw

<!-- BEGIN QUESTION -->

<!-- BEGIN QUESTION -->

**Question 2.2:** Suppose we want to quantify the magnitude and direction of the difference between the estimate and the true height. What calculation or statistic might we perform on these variables to get this information? How can we turn this information into a good test statistic for comparing the two conditions?

<!--
BEGIN QUESTION
name: q2_2
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



<!-- END QUESTION -->

For this project, we will use the error ratio:

$$\Large
\frac{\text{estimate} - \text{true height}}{\text{true height}}
$$

This will give us the magnitude of the difference relative to the true height, to correct for any scale differences based on the actual height of the figure, and whether the estimate was above or below the actual height.

**Question 2.3:** Add an `ErrorRatio` column to `exp1_raw` with the error ratio using the formula above. Store the resulting table as `exp1`.

<!--
BEGIN QUESTION
name: q2_3
points: 1
-->

In [ ]:
error_ratio = ...
exp1 = ...
exp1

In [ ]:
grader.check("q2_3")

Now that we know how we can quantify the difference between the estimate and true height, how can we determine if the differences between the two are drawn from different distributions based on the condition?

As you will recall from Data 8, we can determine whether or not the down-condition errors and the up-condition errors are drawn from the same underlying distribution using an A/B test, in which we shuffle the conditions and calculate some test statistic.

<!-- BEGIN QUESTION -->

<!-- BEGIN QUESTION -->

**Question 2.4:** What are the null and alternative hypotheses for our A/B test?

<!--
BEGIN QUESTION
name: q2_4
manual: true
-->

**Null hypothesis:** _Type your answer here, replacing this text._

**Alternative hypothesis:** _Type your answer here, replacing this text._

<!-- END QUESTION -->



<!-- END QUESTION -->

Before we get into performing the A/B test, we need to create a way for us to easily calculate the error ratios for each iteration.

**Question 2.5:** Fill in the function `add_error_ratio` which will take in a table, create the `ErrorRatio` column as above, and return a new table.

<!--
BEGIN QUESTION
name: q2_5
-->

In [ ]:
def add_error_ratio(tbl):
    """Adds error ratio column to tbl"""
    error_ratio = ...
    tbl = ...
    return tbl

In [ ]:
grader.check("q2_5")

In order to run an A/B test, we need to choose a test statistic. Recall that a good test statistic indicates in the direction of the alternative hypothesis at very high or very low values. In this case, we will use the mean error ratio for each group, which indicates the alternative at higher values. 

The function `calc_abs_diff_mean_error_ratio` defined below will calculate the error ratios, compute the mean for each group, and return the absolute difference between the two. *Note that it requires your `add_error_ratio` function to work correctly.*

In [29]:
def calc_abs_diff_mean_error_ratio(tbl):
    """Calculates absolute difference between mean error ratios"""
    tbl = add_error_ratio(tbl)
    grouped = tbl.group("Condition", np.mean)
    return abs(grouped.column("ErrorRatio mean").item(0) - grouped.column("ErrorRatio mean").item(1))

**Question 2.6:** In the cell below, fill in the code to run the A/B test on our data and collect the test statistic values in `ratio_diffs`. Recall that to run an A/B test we need to shuffle the values of the `Condition` column, drop the original column, and add in the new values.

<!--
BEGIN QUESTION
name: q2_6
points: 1
-->

In [ ]:
ratio_diffs = make_array()

for i in np.arange(1000):
    conditions = ...
    shuffled_exp1 = ...
    ratio_diffs = np.append(ratio_diffs, ...)
    
ratio_diffs[0:5]

In [ ]:
grader.check("q2_6")

To determine the results of our A/B test, we need to calculate a p-value. To calculate the p-value, we find the percentage of results wherein the test statistic was *further in the direction of the alternative hypothesis* than the observed value.

**Question 2.7:** Calculate the observed value of the test statistic using the `calc_abs_diff_mean_error_ratio` and store it as `observed_value`. Then calculate the p-value and store it as `p_value`.

<!--
BEGIN QUESTION
name: q2_7
-->

In [ ]:
observed_value = ...
p_value = ...
p_value

In [ ]:
grader.check("q2_7")

<!-- BEGIN QUESTION -->

<!-- BEGIN QUESTION -->

**Question 2.8:** Using the conventional p-value cutoff of 0.05, which hypothesis do we adopt?

<!--
BEGIN QUESTION
name: q2_8
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



<!-- END QUESTION -->

## Part 3: Height Perceptions and the Game

In this section of the project, we will look at the second experiment from Thomas and Pemstein (2015). In this experiment, the placement of cameras is used to determine what effect the perception of height has on the leader-follower behavior of subjects.

<div class="alert alert-warning">
    
Take a moment to read the <a href="https://www.frontiersin.org/articles/10.3389/fpsyg.2015.00306/full#h4" target="_blank">Experiment 2 methodology</a> from the paper. Ensure you understand the experiment before moving on.

</div>

The payoff structure of the game is reproduced below for your reference.

<img src="exp2-payoffs.jpg" width="600px" alt="Payoff Structure" />

**Note:** In the rest of this lab, <img src="chainmail.png" style="display: inline-block; vertical-align: text-bottom;" width="30px"/> will be referred to as the "chainmail pattern" and <img src="bricks.png" style="display: inline-block; vertical-align: text-bottom;" width="30px"/> as the "brick pattern."

**Question 3.1:** If Player 1 chooses the chainmail pattern and Player 2 the brick pattern, what is the payoff for each player?

<ol type="A" style="list-style-type: lower-alpha;">
    <li>1: \$0, 2: \$2</li>
    <li>1: \$2, 2: \$3</li>
    <li>1: \$3, 2: \$2</li>
    <li>1: \$0, 2: \$0</li>
</ol>

Assign the letter corresponding to your answer to `q3_1` below.

<!--
BEGIN QUESTION
name: q3_1
-->

In [ ]:
q3_1 = ...

In [ ]:
grader.check("q3_1")

**Question 3.2:** What is meant by the "asymmetrical condition" in the paper?

<ol type="A" style="list-style-type: lower-alpha;">
    <li>cameras are placed on different sides (left/right) of the monitor</li>
    <li>one camera is up or down and the other is in the center of the monitor</li>
    <li>one camera is up and the other is down</li>
    <li>both cameras are on the same side of the monitor (both up or both down)</li>
</ol>

Assign the letter corresponding to your answer to `q3_2` below.

<!--
BEGIN QUESTION
name: q3_2
-->

In [ ]:
q3_2 = ...

In [ ]:
grader.check("q3_2")

Now we'll read in the data from the experiment. The `Condition` column here indicates whether the subjects are in the asymmetrical (`1`) or symmetrical (`2`) condition. The `Choice` column indicates their choice of payoff (for themselves) and the `Winnings` column what they actually won.

In [44]:
exp2 = Table.read_table("frontiers2.csv")
exp2

<!-- BEGIN QUESTION -->

<!-- BEGIN QUESTION -->

**Question 3.3:** In the asymmetrical condition, what do you expect the effect of the up/down placement of the camera to have on the winnings? Why?

<!--
BEGIN QUESTION
name: q3_3
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



<!-- END QUESTION -->

In order to determine whether the camera placement had an effect on the payoff choice in the experimental condition, we need to check that people in the one condition chose the payoff-maximizing choice more often that people in the other condition. Let's start by limiting the data to subjects in the experimental condition and creating a variable that indicatings whether the subject chose the payoff-maximizing value.

**Question 3.4:** Filter `exp2` for rows in the experimental condition (`Condition` is 1) and store this as `exp_condition`. Then add a column to `exp_condition` that indicates whether the subject chose the payoff-maximizing value by applying the provided `payoff_maximizing` function to the `Choice` column; store this as the `Payoff` column.

<!--
BEGIN QUESTION
name: q3_4
-->

In [ ]:
def payoff_maximizing(val):
    return int(val == 3)

exp_condition = ...
payoffs = exp_condition.apply(...)
exp_condition = ...
exp_condition

In [ ]:
grader.check("q3_4")

Now that we know which subjects chose the payoff-maximizing value, we can run an A/B test on this variable to determine whether the `Room` variable (which indicates whether the camera was up or down) has any effect on the outcome. We'll use the absolute difference between the proportion of 1's in each group as our test statistic. The provided function `payoff_proportion` will calculate this value when provided an array of `Payoff` values.

In [50]:
def payoff_proportion(payoffs):
    return np.sum(payoffs) / len(payoffs)

payoff_proportion(make_array(0, 1, 0, 1))

Now that we know how to calculate the test statistic, let's think about how we can group our table for the A/B test. We'll need to group on the `Room` variable, and we can use our `payoff_proportion` function as the collection function for `Table.group`.

In [51]:
grouped_exp = exp_condition.group("Room", payoff_proportion)
grouped_exp

Now that we have the grouped values, we can calculate our test statistic by taking the absolute difference of the two values.

In [52]:
payoff_props = grouped_exp.column("Payoff payoff_proportion")
abs(payoff_props.item(0) - payoff_props.item(1))

With the logic for how we calculate the test statistic value in place, let's write a function that will do this for us in each loop of the A/B test.

**Question 3.5:** Fill in the function `calc_condition_test_stat` below which runs through the logic above for a provided table `tbl`.

_Hint:_ You can check that your function is correct by comparing the cell's output to the last code cell.

<!--
BEGIN QUESTION
name: q3_5
-->

In [ ]:
def calc_condition_test_stat(tbl):
    grouped_exp = ...
    payoff_props = ...
    return ...
    
calc_condition_test_stat(exp_condition)

In [ ]:
grader.check("q3_5")

Now that we have an easy way to calculate the test statistic, let's build our A/B test. Recall that we will need to shuffle the values of the `Room` variable and then run `calc_condition_test_stat` on the resulting table.

**Question 3.6:** Fill in the code below to run the A/B test and collect our test statistic values in `condition_stat_values`.

<!--
BEGIN QUESTION
name: q3_6
-->

In [ ]:
condition_stat_values = make_array()
for i in np.arange(1000):
    conditions = ...
    shuffled_exp_condition = ...
    stat_value = ...
    condition_stat_values = ...
    
condition_stat_values[0:5]

In [ ]:
grader.check("q3_6")

Now let's calculate our p-value. As with before, values *higher* than the observed value indicate the alternative hypothesis, so we'll be looking for those again.

**Question 3.7:** Fill in the code below to store the observed value as `condition_observed_value` and the p-value as `condition_p_value`.

<!--
BEGIN QUESTION
name: q3_7
-->

In [ ]:
condition_observed_value = ...
condition_p_value = ...
condition_p_value

In [ ]:
grader.check("q3_7")

<!-- BEGIN QUESTION -->

<!-- BEGIN QUESTION -->

**Question 3.8:** Using the conventional p-value cutoff of 0.05, which hypothesis do we adopt? What does this indicate about our initial question, whether and how the placement of the camera affects how subjects play the game?

<!--
BEGIN QUESTION
name: q3_8
manual: true
-->

_Type your answer here, replacing this text._

**Question 3.9:** Suppose we were to play the game described in this section like we do the prisoner's dilemma, in matches and tournaments. What do our results tell you about strategies for playing this game (amongst real people, not computers)? Describe another strategy, besides camera placement, that could help you win the game. Justify your answer.

<!--
BEGIN QUESTION
name: q3_9
manual: true
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



<!-- END QUESTION -->

---

### References

Thomas, L. E., & D. Pemstein (2015). "What you see is what you get: webcam placement influences perception and social coordination." _Frontiers in Psychology._ https://www.frontiersin.org/articles/10.3389/fpsyg.2015.00306/full

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before     running the cell below, so that all images/graphs appear in the output. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export.
grader.export("proj02.ipynb")